In [ ]:
import os
from dotenv import load_dotenv
import psycopg2

# Load environment variables from .env file
load_dotenv()

def get_connection():
    """Establishes a secure connection using environment variables."""
    return psycopg2.connect(
        dbname=os.getenv("DB_NAME"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        host=os.getenv("DB_HOST"),
        port=os.getenv("DB_PORT")
    )

conn = get_connection()
cursor = conn.cursor()


In [ ]:
import psycopg2

conn = get_connection()
cursor = conn.cursor()

# Enable pgvector extension
cursor.execute("CREATE EXTENSION IF NOT EXISTS vector;")

# Create table for Bible embeddings
cursor.execute("""
    CREATE TABLE IF NOT EXISTS bible_embeddings (
        id SERIAL PRIMARY KEY,
        verse TEXT NOT NULL,
        text TEXT NOT NULL,
        embedding vector(1024) -- Adjust dimension based on your model
    );
""")

# Create table for Shakespeare embeddings
cursor.execute("""
    CREATE TABLE IF NOT EXISTS shakespeare_embeddings (
        id SERIAL PRIMARY KEY,
        ref INTEGER NOT NULL,
        sentence TEXT NOT NULL,
        embedding vector(1024) -- Adjust dimension based on your model
    );
""")

conn.commit()
conn.close()

print("✅ PostgreSQL tables for Bible & Shakespeare embeddings created successfully!")


✅ PostgreSQL tables for Bible & Shakespeare embeddings created successfully!


In [3]:
import ollama
import numpy as np

# Function to get   embeddings using Ollama (with error handling)
def get_embedding(text):
    try:
        embedding_data = ollama.embeddings(model="mxbai-embed-large", prompt=text)
        embedding_data = embedding_data["embedding"]  # Extract embedding
#        embedding_data = np.array(embedding_data)
        return embedding_data
    except Exception as e:
        print(f"❌ Unexpected error generating embedding: {e}")
    return None  # Return None if an error occurs


In [1]:
import sqlite3
def get_bible_text():
    conn = sqlite3.connect("bible.db")
    cursor = conn.cursor()
    
    cursor.execute("SELECT ref, text FROM bible_verses")
    result = cursor.fetchall()
    conn.close()
    return result


In [20]:
import sqlite3
def get_shakespeare_text():
    conn = sqlite3.connect("shakespeare.db")
    cursor = conn.cursor()
    
    cursor.execute("SELECT id, sentence FROM shakespeare_sentences")
    result = cursor.fetchall()
    conn.close()
    return result


In [ ]:
import psycopg2

# Function to insert Bible embeddings into PostgreSQL
def store_bible_embeddings(bible_texts):
    print(f"🔄 Storing Bible embeddings in PostgreSQL... {len(bible_texts)}")
    conn = get_connection()
    cursor = conn.cursor()

    total = len(bible_texts)
    for i, (verse, text) in enumerate(bible_texts, start=1):
        embedding = get_embedding(text)
        cursor.execute(
            "INSERT INTO bible_embeddings (verse, text, embedding) VALUES (%s, %s, %s);",
            (verse, text, embedding)
        )

        if i % 1000 == 0 or i == total:
            percent_done = (i / total) * 100
            print(f"🔄 Inserted {i}/{total} Bible verses ({percent_done:.2f}% done)")
        conn.commit()

    conn.close()
    print("✅ Bible embeddings stored in PostgreSQL!")
bible_texts = get_bible_text()  # Fetches (verse, text)
store_bible_embeddings(bible_texts)


🔄 Storing Bible embeddings in PostgreSQL... 31102
🔄 Inserted 1000/31102 Bible verses (3.22% done)
🔄 Inserted 2000/31102 Bible verses (6.43% done)
🔄 Inserted 3000/31102 Bible verses (9.65% done)
🔄 Inserted 4000/31102 Bible verses (12.86% done)
🔄 Inserted 5000/31102 Bible verses (16.08% done)
🔄 Inserted 6000/31102 Bible verses (19.29% done)
🔄 Inserted 7000/31102 Bible verses (22.51% done)
🔄 Inserted 8000/31102 Bible verses (25.72% done)
🔄 Inserted 9000/31102 Bible verses (28.94% done)
🔄 Inserted 10000/31102 Bible verses (32.15% done)
🔄 Inserted 11000/31102 Bible verses (35.37% done)
🔄 Inserted 12000/31102 Bible verses (38.58% done)
🔄 Inserted 13000/31102 Bible verses (41.80% done)
🔄 Inserted 14000/31102 Bible verses (45.01% done)
🔄 Inserted 15000/31102 Bible verses (48.23% done)
🔄 Inserted 16000/31102 Bible verses (51.44% done)
🔄 Inserted 17000/31102 Bible verses (54.66% done)
🔄 Inserted 18000/31102 Bible verses (57.87% done)
🔄 Inserted 19000/31102 Bible verses (61.09% done)
🔄 Inserted 2

In [ ]:

# Function to insert Shakespeare embeddings into PostgreSQL
def store_shakespeare_embeddings(shakespeare_sentences):
    conn = get_connection()
    cursor = conn.cursor()

    total = len(shakespeare_sentences)
    for i, (ref, sentence) in enumerate(shakespeare_sentences, start=1):
        embedding = get_embedding(sentence)
        cursor.execute(
            "INSERT INTO shakespeare_embeddings (ref, sentence, embedding) VALUES (%s, %s, %s);",
            (ref, sentence, embedding)
        )

        if i % 1000 == 0 or i == total:
            percent_done = (i / total) * 100
            print(f"🔄 Inserted {i}/{total} Shakespeare sentences ({percent_done:.2f}% done)")

    conn.commit()
    conn.close()
    print("✅ Shakespeare embeddings stored in PostgreSQL!")

# Load data from SQLite and store in PostgreSQL
shakespeare_texts = get_shakespeare_text()  # Fetches (sentence,)

store_shakespeare_embeddings(shakespeare_texts)


✅ Bible embeddings stored in PostgreSQL!
🔄 Inserted 1000/70307 Shakespeare sentences (1.42% done)
🔄 Inserted 2000/70307 Shakespeare sentences (2.84% done)
🔄 Inserted 3000/70307 Shakespeare sentences (4.27% done)
🔄 Inserted 4000/70307 Shakespeare sentences (5.69% done)
🔄 Inserted 5000/70307 Shakespeare sentences (7.11% done)
🔄 Inserted 6000/70307 Shakespeare sentences (8.53% done)
🔄 Inserted 7000/70307 Shakespeare sentences (9.96% done)
🔄 Inserted 8000/70307 Shakespeare sentences (11.38% done)
🔄 Inserted 9000/70307 Shakespeare sentences (12.80% done)
🔄 Inserted 10000/70307 Shakespeare sentences (14.22% done)
🔄 Inserted 11000/70307 Shakespeare sentences (15.65% done)
🔄 Inserted 12000/70307 Shakespeare sentences (17.07% done)
🔄 Inserted 13000/70307 Shakespeare sentences (18.49% done)
🔄 Inserted 14000/70307 Shakespeare sentences (19.91% done)
🔄 Inserted 15000/70307 Shakespeare sentences (21.34% done)
🔄 Inserted 16000/70307 Shakespeare sentences (22.76% done)
🔄 Inserted 17000/70307 Shakespe

In [ ]:
def find_similar_bible_verses(shakespeare_sentence, top_k=5):
    embedding = get_embedding(shakespeare_sentence)  # Get embedding as a list

    conn = get_connection()
    cursor = conn.cursor()

    query = """
    SELECT verse, text, 1 - (embedding <=> %s) AS similarity
    FROM bible_embeddings
    ORDER BY embedding <=> %s
    LIMIT %s;
    """
    cursor.execute(query, (embedding, embedding, top_k))
    results = cursor.fetchall()
    conn.close()

    return results

# Example Query
query_text = "To be, or not to be, that is the question."
bible_matches = find_similar_bible_verses(query_text)

print(f"\n📖 **Bible Verses Similar to Shakespeare's Quote: {query_text}**")
for verse, text, similarity in bible_matches:
    print(f"{verse} - {text} (Similarity: {similarity:.4f})")



📖 **Bible Verses Similar to Shakespeare's Quote: To be, or not to be, that is the question.**
Co2 8:12 - For if there be first a willing mind, it is accepted according to that a man hath, and not according to that he hath not.~ (Similarity: 0.6588)
Mat 11:3 - And said unto him, Art thou he that should come, or do we look for another?~ (Similarity: 0.6555)
Kg2 2:10 - And he said, Thou hast asked a hard thing: nevertheless, if thou see me when I am taken from thee, it shall be so unto thee; but if not, it shall not be so.~ (Similarity: 0.6362)
Job 39:9 - Will the unicorn be willing to serve thee, or abide by thy crib?~ (Similarity: 0.6322)
Gal 1:10 - For do I now persuade men, or God? or do I seek to please men? for if I yet pleased men, I should not be the servant of Christ.~ (Similarity: 0.6302)


In [ ]:
import psycopg2

def find_similar_bible_verses_cosine(shakespeare_sentence, top_k=5):
    embedding = get_embedding(shakespeare_sentence)  # Get embedding as a list
    conn = get_connection()
    cursor = conn.cursor()

    cursor.execute("""
        SELECT verse, text, embedding <=> %s AS similarity
        FROM bible_embeddings
        ORDER BY similarity DESC
        LIMIT %s;
    """, (str(embedding), top_k))

    # Fetch and print results
    results = cursor.fetchall()
    return results


query_text = "To be, or not to be, that is the question."
bible_matches = find_similar_bible_verses_cosine(query_text)

for result in bible_matches:
    print(f"ID: {result[0]}, Name: {result[1]}, Cosine Similarity: {result[2]}")

ID: Num 31:10, Name: And they burnt all their cities wherein they dwelt, and all their goodly castles, with fire.~, Cosine Similarity: 0.7576054290461446
ID: Est 9:6, Name: And in Shushan the palace the Jews slew and destroyed five hundred men.~, Cosine Similarity: 0.7569403794042351
ID: Act 18:17, Name: Then all the Greeks took Sosthenes, the chief ruler of the synagogue, and beat him before the judgment seat. And Gallio cared for none of those things.~, Cosine Similarity: 0.7554121530553088
ID: Deu 3:6, Name: And we utterly destroyed them, as we did unto Sihon king of Heshbon, utterly destroying the men, women, and children, of every city.~, Cosine Similarity: 0.750413240364846
ID: Deu 2:34, Name: And we took all his cities at that time, and utterly destroyed the men, and the women, and the little ones, of every city, we left none to remain:~, Cosine Similarity: 0.7503269107409909


In [11]:
conn = psycopg2.connect(dbname="bible", user="bible",
                        password="bible", host="localhost", port="5432")
cursor = conn.cursor()
cursor.execute("""
    CREATE INDEX ON bible_embeddings USING ivfflat (embedding vector_l2_ops);
""")
conn.commit()
